In [17]:
from pprint import pprint
import praw
from praw.models import MoreComments
import requests
import json
import numpy as np
import pandas as pd
from collections import Counter
from math import ceil
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams

import nltk.data
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
from nltk.tokenize import word_tokenize

from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten

from math import floor
import copy

from submission_corpus import SubmissionCorpus
from ngram_generator import NgramCommentGenerator

In [32]:
s = 'abc'
s.replace('a','')

'bc'

In [18]:
client_id = '71ZX5Cupn2Ohpg'
client_secret = 'nzCz5_WlQM4LbJxX-t_3m-tPgZw'

reddit = praw.Reddit(user_agent='Comment Extraction',client_id=client_id, client_secret=client_secret)
subreddit = reddit.subreddit('AmITheAsshole')

judgement_categories = ['YTA', 'NTA', 'ESH', 'NAH', 'INFO']

In [19]:
sc = SubmissionCorpus(reddit.submission(id='dv9ogm'), bfs_depth=1, judgement_categories=judgement_categories, judgement_weight='none')
cc = sc.get_commentCorpus()

In [20]:
ncg = NgramCommentGenerator(cc, N=4)

In [12]:
ncg.generate_comment('YTA')

"<c> <s> YTA . </s> <s> I think your desire for a child does not outweigh your SIL 's desire to `` keep it in the family . </s> <s> I get that you can ’ t get pregnant yourself but can you at least think about the emotional toll it would be to get pregnant with your brother ’ s child , carry it to term , GO THROUGH BIRTH , and then no matter if your feelings change and transform the child YOU carried is then taken from you to be raised by your brother and sister in law feels or reacted are unnecessary . </s> <s> I puked multiple times a day every day for seven months with nothing being wrong during my very wanted pregnancy . </s> <s> A get reasonable boundary to have when she 's currently upset . </s> <s> To ask this of your sister just because he wants to keep it “ in the blood ” is fucking creepy and crosses so many boundaries . </s> </c>"

In [24]:
class Submission_Corpus:
    def __init__(self, submission, bfs_depth=2, judgement_categories=None, judgement_weight='upvotes'):
        self.comment_meta = []
        self.judgement_categories = judgement_categories
        self.judgement_weight = judgement_weight
        self.judgements = {}
        for category in self.judgement_categories:
            self.judgements[category] = []

        self.submission = submission
        self.original_post = submission.selftext
        self.comment_forrest = self.submission.comments
        self.comment_bfs(bfs_depth=bfs_depth)       
        
    def comment_bfs(self, bfs_depth=0):
        # Initialize queue to hold comment sub trees during BFS.
        bfs_queue = []
        
        # Populate queue with first level of comments.
        for comment in self.comment_forrest:
            bfs_queue.append(comment)
            
        current_level_size = len(bfs_queue)
        next_level_size = 0
        level_count = 0
        
        while (len(bfs_queue) > 0) and (level_count < bfs_depth):
            comment = bfs_queue.pop(0)
            current_level_size -= 1
            next_level_size += 1
            
            comment_features = None
            try:
                comment_features = self.extract_comment(comment)
            except:
                pass

            if comment_features is not None:
                self.comment_meta.append(comment_features)
                for reply in comment.replies:
                    bfs_queue.append(reply)
            
            if current_level_size == 0:
                current_level_size = next_level_size
                level_count += 1
            
    def extract_comment(self, comment, judgement_extraction_method='prefix', judgement_weighting='upvotes'):
        judgement = self.extract_judgement(comment.body, extraction_method=judgement_extraction_method)
        score = comment.score if judgement_weighting=='upvotes' else 1
        self.judgements[judgement].append(score)
        body = self.tokenize_comment(comment.body)
        comment_features = {
            'id' : comment.id,
            'author': comment.author,
            'body': body,
            'score' : score,
            'judgement': judgement
        }
        return(comment_features)
    
    def extract_judgement(self, txt, extraction_method='prefix'):
        if extraction_method == 'prefix':
            for category in self.judgement_categories:
                if txt[:len(category)] == category:
                    return(category)
    
    def summarize_judgement(self):
        total_judgements = sum([sum(count) for count in self.judgements.values()])
        judgement_summary = [(category, sum(count)/total_judgements) for category, count in self.judgements.items()]
        return(judgement_summary)
    
    def get_judgement_summary(self):
        return(self.judgement_summary)
    
    def tokenize_sentence(self, sent):
        try:
            tokenized_sent = word_tokenize(sent)
        except:
            tokenized_sent = []
        tokenized_sent.insert(0, '<s>')
        tokenized_sent.append('</s>')
        return(tokenized_sent)
    
    def tokenize_comment(self, comment_txt):
        sentences = sent_detector.tokenize(comment_txt.strip())
        tokenized_sentences = [self.tokenize_sentence(sent) for sent in sentences]
        tokenized_comment = ['<c>']
        for sent in tokenized_sentences:
            tokenized_comment += sent
        tokenized_comment += ['</c>']
        return(tokenized_comment)
        
    def get_commentCorpus(self):
        comments_by_category = {}
        for category in self.judgement_categories:
            comments_by_category[category] = []
        
        
        for comment in self.comment_meta:
            if self.judgement_weight == 'upvotes':
                for i in range(max(5,floor(comment['score']/50))):
                    comments_by_category[comment['judgement']].append(comment['body'])
                    
            else:
                comments_by_category[comment['judgement']].append(comment['body'])            
                
        return(comments_by_category)
    
    def get_simpleWeightedCorpus(self):
        comments_by_category = {}
        for category in self.judgement_categories:
            comments_by_category[category] = []
        
        for comment in self.comment_meta:
            comments_by_category[comment['judgement']].append((comment['score'], comment['body']))          
        
        return(comments_by_category)
    
        

In [5]:
SC = Submission_Corpus(reddit.submission(id='dv9ogm'), bfs_depth=1, judgement_categories=judgement_categories, judgement_weight='none')

In [6]:
SC.summarize_judgement()

[('YTA', 0.9961707901927617),
 ('NTA', 0.0),
 ('ESH', 0.0),
 ('NAH', 0.0),
 ('INFO', 0.003829209807238322)]

In [7]:
cc = SC.get_commentCorpus()

In [8]:
swcc = SC.get_simpleWeightedCorpus()
f = open('dv9ogm_corpus.txt', 'w')
f.write(str(swcc))
f.close()

In [9]:
print(cc['YTA'][0])

['<c>', '<s>', 'YTA', '.', '</s>', '<s>', 'I', '’', 'm', 'trying', 'to', 'say', 'that', 'as', 'kindly', 'as', 'possible', 'but', 'why', 'would', 'you', 'think', 'someone', 'who', 'is', 'adamantly', 'against', 'having', 'children', 'would', 'want', 'to', 'carry', 'yours', '?', '</s>', '<s>', 'If', 'your', 'husband', 'is', 'that', 'close', 'to', 'his', 'sister', ',', 'I', 'would', 'have', 'thought', 'that', 'he', 'would', 'know', 'better', 'than', 'to', 'make', 'such', 'a', 'tone', 'deaf', 'request', '.', '</s>', '<s>', 'Also', ',', 'it', 'may', 'be', 'unrelated', ',', 'but', 'the', '“', 'important', 'to', 'keep', 'this', 'surrogacy', 'confined', 'to', 'his', 'blood', '”', 'bit', 'makes', 'me', 'roll', 'my', 'eyes', 'so', 'hard', '.', '</s>', '<s>', 'Your', 'parents', 'are', 'of', 'course', 'only', 'going', 'to', 'see', 'this', 'from', 'your', 'point', 'of', 'view', 'so', 'their', 'opinions', 'on', 'how', 'your', 'sister', 'in', 'law', 'feels', 'or', 'reacted', 'are', 'unnecessary', '.',

In [11]:
for i in ngrams(cc['YTA'][0],3):
    print(i)

('<c>', '<s>', 'YTA')
('<s>', 'YTA', '.')
('YTA', '.', '</s>')
('.', '</s>', '<s>')
('</s>', '<s>', 'I')
('<s>', 'I', '’')
('I', '’', 'm')
('’', 'm', 'trying')
('m', 'trying', 'to')
('trying', 'to', 'say')
('to', 'say', 'that')
('say', 'that', 'as')
('that', 'as', 'kindly')
('as', 'kindly', 'as')
('kindly', 'as', 'possible')
('as', 'possible', 'but')
('possible', 'but', 'why')
('but', 'why', 'would')
('why', 'would', 'you')
('would', 'you', 'think')
('you', 'think', 'someone')
('think', 'someone', 'who')
('someone', 'who', 'is')
('who', 'is', 'adamantly')
('is', 'adamantly', 'against')
('adamantly', 'against', 'having')
('against', 'having', 'children')
('having', 'children', 'would')
('children', 'would', 'want')
('would', 'want', 'to')
('want', 'to', 'carry')
('to', 'carry', 'yours')
('carry', 'yours', '?')
('yours', '?', '</s>')
('?', '</s>', '<s>')
('</s>', '<s>', 'If')
('<s>', 'If', 'your')
('If', 'your', 'husband')
('your', 'husband', 'is')
('husband', 'is', 'that')
('is', 'that'

In [12]:
def make_ngram_corpus(corpus, N=3):
    ngram_tups_list = list(ngrams(corpus, N))
    ngram_corpus = []
    for gi in ngram_tups_list:
        if not gi[0:N-1] in [x[0] for x in ngram_corpus]:
            next_word_dict = {}
            
            for gj in ngram_tups_list:
                if gj[0:N-1] == gi[0:N-1]:
                    if gj[N-1] in next_word_dict.keys():
                        next_word_dict[gj[N-1]] += 1
                    else:
                        next_word_dict[gj[N-1]] = 1
            gi_count = sum(next_word_dict.values())
            next_word_prob_tups = tuple([(key, value/gi_count) for key, value in next_word_dict.items()])
            ngram_corpus.append((gi[0:N-1], gi_count, next_word_prob_tups))
    return(ngram_corpus)
    

In [13]:
flat_comment_corpus = []
for comment in cc['YTA']:
    for token in comment:
        flat_comment_corpus.append(token)
        
ngram_corpus = make_ngram_corpus(flat_comment_corpus,3)

In [14]:
ngram_corpus

[(('<c>', '<s>'), 24, (('YTA', 1.0),)),
 (('<s>', 'YTA'),
  24,
  (('.', 0.625),
   ('Honestly', 0.041666666666666664),
   ('for', 0.041666666666666664),
   ('...', 0.041666666666666664),
   ('-', 0.041666666666666664),
   ('Why', 0.041666666666666664),
   (',', 0.08333333333333333),
   ('You', 0.041666666666666664),
   ('jesus', 0.041666666666666664))),
 (('YTA', '.'), 15, (('</s>', 1.0),)),
 (('.', '</s>'),
  143,
  (('<s>', 0.8741258741258742), ('</c>', 0.1258741258741259))),
 (('</s>', '<s>'),
  156,
  (('I', 0.11538461538461539),
   ('If', 0.03205128205128205),
   ('Also', 0.019230769230769232),
   ('Your', 0.03205128205128205),
   ('At', 0.00641025641025641),
   ('You', 0.14743589743589744),
   ('It', 0.04487179487179487),
   ('Edit', 0.01282051282051282),
   ('And', 0.07051282051282051),
   ('What', 0.05128205128205128),
   ('Did', 0.00641025641025641),
   ('Yes', 0.00641025641025641),
   ('How', 0.02564102564102564),
   ('Is', 0.00641025641025641),
   ('Surrogacy', 0.0064102564

In [15]:
def ngram_generate_comment(corpus, N):
    comment_starts = []
    for gi  in corpus:
        if gi[0][0] == '<c>':
            comment_starts.append(gi)
    num_starts = sum([gi[1] for gi in comment_starts])
    probs=[gi[1]/num_starts for gi in comment_starts]
    comment_start = comment_starts[np.random.choice(list(range(len(comment_starts))), p=probs)]
    comment = [w for w in comment_start[0]]
    while comment[-1] != '</c>':
        prev_gram = tuple(comment[-(N-1):])
        nnext_choices = None
        for gi in corpus:
            if gi[0] == prev_gram:
                next_choices = gi[2]
        if next_choices is not None:
            next_probs = [x[1] for x in next_choices]
            next_word = next_choices[np.random.choice(list(range(len(next_choices))),p=next_probs)][0]
            comment.append(next_word)
        else:
            print('N-Gram not found in corpus')
    return(comment)

In [16]:
for i in range(10):
    comment = ' '.join(ngram_generate_comment(ngram_corpus, 3))
    print(comment)

<c> <s> YTA - I know the I do n't have anyone ever asking that question but damn . </s> <s> That you had to prep her lavishly for the very thing she is so against . </s> <s> I think your desire for wanting children is outweighing your logic for other people ’ s a developmental defect and you and your husband 's desire to `` keep it “ in the family '' ? </s> <s> She 's never had kids . </s> <s> Asking someone to go to work for a while ( great , you told her that you are incredibly selfish for trying to create a win-win '' but you thought that she might not want kids ; hormones are a thing ) , and then to get online to complain about it . </s> <s> Mine worked out well , I am and I know you did n't just ask her . </s> <s> First of all unless you did all of the time a desire to `` keep it “ in the family '' ? </s> </c>
<c> <s> YTA jesus christ , I also have not been able to have when she understandably got angry at you if you 're not using your womb . </s> </c>
<c> <s> YTA . </s> <s> First

In [10]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, GRU, Dense, Activation
from gensim.models import Word2Vec

Using TensorFlow backend.


In [10]:
def build_rnn_corpus(cc, wv, N=10):
    all_comments = []
    all_words = []
    category_inds = np.arange(len(judgement_categories))
    np.random.shuffle(category_inds)

    for category_ind in category_inds:
        comments = cc[judgement_categories[category_ind]]
        comment_inds = np.arange(len(comments))
        np.random.shuffle(comment_inds)
        for comment_ind in comment_inds:
            all_comments.append(comments[comment_ind])
            for token in comments[comment_ind]:
                all_words.append(token)
                
    all_ngrams = ngrams(all_words,N)
    X_train = []
    y_train = []
    for ngram in all_ngrams:
        X_train.append([wv[token] for token in ngram[:-1]])
        y_train.append(wv[ngram[-1]])

    X_train = np.array(X_train)
    y_train = np.array(y_train)
    
    return(X_train, y_train)

In [11]:
all_comments = []
all_words = []
category_inds = np.arange(len(judgement_categories))
np.random.shuffle(category_inds)

for category_ind in category_inds:
    comments = cc[judgement_categories[category_ind]]
    comment_inds = np.arange(len(comments))
    np.random.shuffle(comment_inds)
    for comment_ind in comment_inds:
        all_comments.append(comments[comment_ind])
        for token in comments[comment_ind]:
            all_words.append(token)

word2vec = Word2Vec(all_comments, min_count=1)            

NameError: name 'Word2Vec' is not defined

In [95]:
#all_ngrams = ngrams(all_words,5)
#X_train = []
#y_train = []
#for ngram in all_ngrams:
#    X_train.append([word2vec.wv[token] for token in ngram[:-1]])
#    y_train.append(word2vec.wv[ngram[-1]])
#    
#X_train = np.array(X_train)
#y_train = np.array(y_train)
#X_train = np.array([word2vec.wv[token] for token in all_words])
#y_train

In [96]:
#X_train, y_train = build_rnn_corpus(cc, word2vec.wv)

In [101]:
model = Sequential()
model.add(GRU(units=32, return_sequences=True))
model.add(Dense(100))
model.compile(optimizer='adam', loss='mean_squared_error')

In [102]:
X_train, y_train = build_rnn_corpus(cc, word2vec.wv)
model.fit(X_train[:,:,:], y_train[:,:], shuffle=False, batch_size=16, epochs=1)
    

ValueError: Error when checking target: expected dense_14 to have 3 dimensions, but got array with shape (3649, 100)

In [99]:
y_pred = model.predict(X_train[:0,:,:])

In [100]:
s = [word2vec.wv.similar_by_vector(X_train[0,i,:], topn=1)[0][0] for i in range(X_train.shape[1])]
print(y_pred)
for pred in y_pred:
    print(word2vec.wv.similar_by_vector(pred, topn=1)[0][0])
    s.append(word2vec.wv.similar_by_vector(pred, topn=1)[0][0])
print(s)


[]
['<c>', '<s>', 'INFO', '>', 'My', 'husbands', 'parents', 'are', 'sympathetic']


In [1]:
import tensorflow as tf
import numpy as np
import keras.backend as K
from tqdm import tqdm
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Input, Dense, GRU, TimeDistributed, AveragePooling1D, Flatten
    
class Comment_GRU(object):
    def __init__(self, judgements, comment_corpus, N=4, batch_size=16, epochs=10):    
        self.judgements = judgements
        self.comment_corpus = comment_corpus
        self.w2v = self.build_word2vec()
        self.vectorize_corpus()
        self.N = N
        self.vocab_size = len(self.w2v.wv.vocab)
        self.embed_size = 100
        self.seed = None
        self.batch_size = batch_size
        self.epochs = epochs
        
        self.model = Sequential()
        self.model.add(GRU(units=32, batch_input_shape=(self.batch_size, self.N-1, 100), stateful=True, return_sequences=True))
        #self.model.add(TimeDistributed(Dense(1)))
        #self.model.add(AveragePooling1D())
        self.model.add(Flatten())
        self.model.add(Dense(100))
        self.model.compile(optimizer='adam', loss='mean_squared_error')
        
        for epoch in range(self.epochs):
            print('Epoch: {}'.format(epoch+1))
            Xc,yc = self.build_dataset() 
            X = np.concatenate(list(filter(lambda vals: True if vals.shape[0]>0 else False, Xc.values())),axis=0)
            y = np.concatenate(list(filter(lambda vals: True if vals.shape[0]>0 else False, yc.values())),axis=0)
            for i in range(int(X.shape[0]/self.batch_size)):
                self.model.train_on_batch(X[self.batch_size*i:self.batch_size*(i+1),:,:], y[self.batch_size*i:self.batch_size*(i+1),:])
    
    def apply_batch_windowing(self, c):
        num_subsequences = (c.shape[0]-self.N)+1
        start_row = 0 if num_subsequences > self.batch_size else self.batch_size - num_subsequences
        windowed_c = np.zeros((self.batch_size, self.N-1, 100))
        for i in range(start_row,self.batch_size):
            windowed_c[i,:,:] = c[i:i+self.N-1,:]
        
        return(windowed_c)
    
    def commentMatrix_to_commentSentence(self, comment_mat):
        comment_wordVecs = [comment_mat[0,i,:] for i in range(comment_mat.shape[1]-1)]
        for subsequence_ind in range(comment_mat.shape[0]):
            comment_wordVecs.append(comment_mat[subsequence_ind,-1,:])
        comment_words = [self.w2v.wv.similar_by_vector(wordVec, topn=1)[0][0] for wordVec in comment_wordVecs]
        print(comment_words)
            
    
    def generate_comment(self, judgement, max_len):
        seed = self.apply_batch_windowing(self.comment_corpus[judgement][0])
        comment = None
        for i in range(max_len):
            self.commentMatrix_to_commentSentence(seed)
            #print(seed.shape)
            next_subsequence = self.model.predict(seed)
            print(next_subsequence.shape)
            #print(next_word.shape)
            if comment is None:
                comment = next_subsequence[1:,:].reshape((1,self.N-1,100))
            else:
                comment = np.concatenate((comment,next_subsequence[1:,:].reshape((1,self.N-1,100))), axis=0)
                #print(comment.shape)
            seed = np.concatenate((seed[1:,:,:].reshape((self.batch_size-1,self.N-1,100)),comment[-1,:,:].reshape((1,self.N-1,100))), axis=0).reshape(self.batch_size, self.N-1, 100)
            
        self.commentMatrix_to_commentSentence(comment)
            
    # TO DO: Set p to comment upvote score to change sampling distribution with respect to comment popularity.
    #def build_batch(self, p=None):
        
        
    def build_dataset(self):
        # Use np.random.permutation to create a new ordering of comments for each judgement category. 
        # self.comment_corpus.values() gives the comments for each category, i.e. each element of the list 
        # returned by .values() is the list of comments for a particular category, and using len gives the 
        # number of comments for each category. The function np.arrange creates the default ordering 
        # (0,1,...len(vals)-1), and np.random.permutation gives a new permuted ordering.
        shuffled_comment_inds = list(map(lambda vals: np.random.permutation(np.arange(len(vals))), self.comment_corpus.values()))
        
        # Zip the new comment orderings back to the judgement category labels.
        category_comment_dict = dict(zip(self.comment_corpus.keys(), shuffled_comment_inds))
        
        X = {}
        y = {}
        for category_name, category_comment_inds in category_comment_dict.items():
            X[category_name] = []
            y[category_name] = []
            for comment_ind in category_comment_inds:
                num_subsequences = self.comment_corpus[category_name][comment_ind].shape[0]-self.N
                for i in range(num_subsequences):
                    X[category_name].append(self.comment_corpus[category_name][comment_ind][i:i+self.N-1,:])
                    y[category_name].append(self.comment_corpus[category_name][comment_ind][i+self.N,:])
            X[category_name] = np.array(X[category_name])
            y[category_name] = np.array(y[category_name])
        return(X,y)
    
    def build_word2vec(self):
        all_comments = []
        all_words = []
        category_inds = np.arange(len(self.judgements))
        for category_ind in category_inds:
            for comment_ind in np.arange(len(self.comment_corpus[self.judgements[category_ind]])):
                all_comments.append(self.comment_corpus[self.judgements[category_ind]][comment_ind])
                #for token in self.comment_corpus[self.judgements[category_ind]][comment_ind]:
                #    all_words.append(token)

        return(Word2Vec(all_comments, min_count=1))
        
    def vectorize_corpus(self):
        for category_ind in np.arange(len(self.judgements)):
            comment_inds = np.arange(len(self.comment_corpus[self.judgements[category_ind]]))
            
            for comment_ind in comment_inds:
                token_inds = np.arange(len(self.comment_corpus[self.judgements[category_ind]][comment_ind]))
                
                for token_ind in token_inds:
                    self.comment_corpus[self.judgements[category_ind]][comment_ind][token_ind] = self.w2v.wv[self.comment_corpus[self.judgements[category_ind]][comment_ind][token_ind]]
                
                self.comment_corpus[self.judgements[category_ind]][comment_ind] = np.array(self.comment_corpus[self.judgements[category_ind]][comment_ind])


Using TensorFlow backend.


In [2]:
cc_copy = copy.deepcopy(cc)
c_gru = Comment_GRU(judgement_categories, cc_copy, N=32, batch_size=2, epochs=2 )

NameError: name 'copy' is not defined

In [ ]:
c_gru.generate_comment('YTA', 20)

In [29]:
class Subreddit_Corpus:
    def __init__(self, subreddit=None, retrieval_limit=None, bfs_depth=3, judgement_categories=None):
        self.bfs_depth = bfs_depth
        self.judgement_categories = judgement_categories
        self.subreddit = subreddit
        self.corpus = []
        self.submissions = []
        
        for submission in self.subreddit.new(limit=retrieval_limit):
            self.submissions.append(Submission_Corpus(submission, bfs_depth=bfs_depth, judgement_categories=judgement_categories))
            print(submission.selftext)
    
    def extract_submission(self, submission):
        sub = Submission_Corpus(submission, bfs_depth=self.bfs_depth, judgement_categories=self.judgement_categories)

In [30]:

SS = Subreddit_Corpus(subreddit=subreddit, retrieval_limit=5, judgement_categories=judgement_categories)

So for a little backstory I (18 f) met my now ex friend who I'll call Dave (26 m) about seven months ago, everything was fine until his gf (23) of four years broke up with him, he came to me for help and i was there for him.

 Nothing was unusual until about two weeks after the break up when he asked me a favor. The favor was to masturbate with him, which I politely declined, and nothing more was said about that until a week later when he asked me again,  I politely declined again, but he kept on asking me and begging me. It then went from him asking me that, to him begging me to leave my bf for him, which i kept telling him no.

I then told my brother what he was doing and showed him screenshots. My brother and my bf tell me call him so that my brother can confront Dave, I finally give in, I call him and give my brother the phone. My brother confronts him for the harassment, I don't like conflict so go outside and let my brother deal with Dave so i don't know what was said on either e